## Statistical Methods for the Physical Sciences (5214SMFA3Y)
## Individual mini-Project -- Model fitting and hypothesis testing: the search for WIMPs
### Timo Halbesma, 6126561, 2016/01/08

In [ ]:
# Enable showing plots inside iPython notebook
%matplotlib inline

In [ ]:
import scipy
from scipy import stats
import numpy
import matplotlib
from matplotlib import pyplot
from matplotlib.ticker import MaxNLocator
import pandas

matplotlib.rcParams.update({'font.size': 22})

### Introduction
Current data from the Fermi gamma-ray observatory is providing hints of the existence of anomalous extended GeV gamma-ray emission at the centre of our galaxy. This GeV continuum emission is causing excitement in the astroparticle-physics community, because it may be associated with the decay of the hitherto-undetected weakly interacting massive particles (WIMPs), which are thought to make up dark matter.

This mini-project is based on analysing simulated spectral data from a hypothetical future gamma-ray observatory, whose main objective is to search for and study the extended dark matter decay signature in the centres of nearby galaxies. The data consists of a list of energies of photons detected in an observation of a nearby galaxy. Your task is to convert this data into a gamma-ray spectrum (this is just a histogram of photon numbers versus energy in discrete energy bins), carry out some simple tests and determine the shape of the continuum (with errors on the model parameters) as well as search for and characterise spectral emission lines.


### Objectives
_Important note_: throughout all of the following you should assume that in addition to any astrophysical source photons with spectra described below, there is an additional instrumental background photon continuum which contributes a constant number of photons per GeV. For the dataset you are given, $C = 1.5$ photons/GeV.

### Question 1
- The simple alternative to dark matter decay is that the continuum is produced by the combined unresolved emission from a large number of gamma-ray pulsars in the centre of the target galaxy. Assume that the spectrum of gamma-ray pulsars is a simple power- law following the relation: $$ dN = N_0 \left( \frac{E}{E_0} \right)^{-\Gamma} dE $$ where in our formalism, $dN$ is the number of photons expected in the infinitesimal energy range $dE$, and $N_0$ is the normalisation of the spectrum (in photons/GeV) at a fixed photon energy $E_0$. $\Gamma$ is the power-law index and is also known as the 'photon index'. Assuming that $\Gamma = 2$, use a _non-parametric significance test_ to compare the shape of this pulsar spectrum with that of your data (note that you don't need to know $N_0$!).

### Answer 1

### Question 2
- Fit the observed spectrum with continuum models and find the best-fitting model. You should compare the following three possibilities:
    - A simple power-law.
    - A broken power-law:
    $$ dN = N_0 \left( \frac{E}{E_0} \right)^{-\Gamma_1} dE \quad \rm{ for } E \leq E_{\rm bk} $$ $$ dN = N_0 \left( \frac{E_{\rm bk}}{E_0} \right)^{-\Gamma_1} \left( \frac{E}{E_{\rm bk}} \right)^{-\Gamma_2} dE \quad \rm{ for } E > E_{\rm bk} $$ where $E_{\rm bk}$ is the break energy and $\Gamma_1$ and $\Gamma_2$ denote respectively the photon indices below and above the break energy.
    - An exponentially cut-off power-law:
    $$ dN = N_0 \left( \frac{E}{E_0}\right)^{- \Gamma} \exp\left(-E/E_{\rm cut}\right) dE $$ where $E_{\rm cut}$ is the cut-off energy.

### Answer 2

In [ ]:
def parse_and_clean_dataset(filename="Halbesma_energies.txt"):
    ''' read data from file
        clean is not needed; file only contains one column with .2f '''
    energies = numpy.genfromtxt(filename, skiprows=0)
    
    return energies


def inspect_data(energies):
    ''' Inspect the data to see the effect of the chosen binsize.
        NB: not correct for background, bins with <20 counts are not cut '''
    
    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = pyplot.subplots(3, 2, figsize=(12, 18))
    axes = [ax1, ax2, ax3, ax4, ax5, ax6]
    
    # Use a different set of 'best'-practice number of bins (equally spaced binsizes)
    # htt/s://en.wikipedia.org/wiki/Histogram#Number_of_bins_and_width
    N = len(energies)
    # Freedman–Diaconis rule
    h = (2*numpy.subtract(*numpy.percentile(energies, [75, 25]))*N**(-1./3))
    FD = numpy.ceil((numpy.max(energies) - numpy.min(energies)) / h)
    sturges = numpy.ceil(numpy.log2(N)+1)  # assumes normal distribution
    rice = numpy.ceil(2*N**(1./3))
    
    bins = [sturges, rice, FD, int(numpy.sqrt(N)), 500, 700]
    labels = [r'$\log (2)+1$', r'$2*N^{1/3}$', r'FD', r'$\sqrt{N}$', "fixed", "fixed"]
    for i, (ax, nr_of_bins) in enumerate(zip(axes, bins)):
        counts, edges = numpy.histogram(energies,
            range=[numpy.min(energies), numpy.max(energies)],
            bins=nr_of_bins, density=False)
        
        ax.plot(edges[1:], counts, lw=3, ls="steps-mid", label=labels[i])
        ax.axhline(20, c="r", lw=2, ls="dashed")
    
        ax.set_title("nr_of_bins = {0}".format(nr_of_bins))
        ax.set_ylabel("Counts")
        ax.set_xlabel("E (GeV)")
        ax.set_yscale("log")
        ax.legend()
    
    fig.tight_layout()
    pyplot.show()

** Binning the data **

The first step is to visually inspect the data, and try to determine an optimal number of bins. We set a hard limit on the minimum number of counts in each bin. If the number of counts is higher than twenty, then we can assume the error on the counts in the bins is normally distributed and we can use the $\chi^2$ statistic later on for our fitting routines. The optimal number of bins is not trivial, but some formulae are available which indicate the optimal binsize, or the optimal number of bins as a function of the sample size. We use several to inspect the data.

In [ ]:
if __name__ == '__main__':
    energies = parse_and_clean_dataset()
    inspect_data(energies)

In the above plot the red dashed horizontal line indicates the twenty count cut-off. We observe that underbinning results in missing features in the low-energy part of the spectrum, whereas over-binning leads to cutting the tail of the distribution. This will be a problem later on, because we will fit three different models. Should we cut the data above the break- or cut-off energy we will effectively only fit one model (the simple power law). We really want to have the features in the low-energy part of the spectrum in addition to not-igonoring the tail of the distribution.

The solution to this problem is to use unequally spaced bins, which according to dr. Uttley is perfectly fine. This way we will get a roughly equal count rate in each bin.

In [ ]:
def bin_data_adaptive(energies, adaptive=True):
    ''' bin data in unevenly spaced bins --> count rate is roughly equal over all x '''

    # Fit routines should like smaller numbers better *_*
    energies = energies / (numpy.mean(energies))

    if adaptive:
        # Bin data using 'adaptive' bin sizes
        counts, edges = numpy.histogram(energies,
            range=[numpy.min(energies), numpy.max(energies)],
            bins=numpy.percentile(energies, range(0, 101, 2)), density=False)
    else:
        N = len(energies)
        # Freedman–Diaconis rule
        h = (2*numpy.subtract(*numpy.percentile(energies, [75, 25]))*N**(-1./3))
        FD = numpy.ceil((numpy.max(energies) - numpy.min(energies)) / h)
        counts, edges = numpy.histogram(energies,
            range=[numpy.min(energies), numpy.max(energies)],
            bins=FD, density=False)
    
    # Calculate the binsizes (which is different for each bin!)
    binsizes = numpy.zeros(len(counts), dtype=numpy.float64)
    for i in range(0, len(edges)-1):
        binsizes[i] = edges[i+1] - edges[i]  # in GeV

    # Cut the bins with less than 20 counts --> Poisson
    # NB using adaptive method there should not be bins with less than 20 counts (e.g. in the tail)
    counts = counts[counts > 20]
    # Reduce length of edges and binsize too
    edges = edges[0:len(counts) + 1]  # +1 because len(edges) = len(counts) +1
    binsizes = binsizes[0:len(counts)]

        
    # Calculate intrinsic error of bin. 
    # NB this must be done before correcting for background!
    # We assume --> err = sqrt(N) is normally distributed (since counts > 20)
    sample_size = len(energies)
    err = numpy.sqrt(counts) / (sample_size * binsizes)
    
    # Correct for background counts: C = 1.5 photons/GeV
    counts = counts - (1.5 * binsizes)
  
    # Calculate density
    dens = counts / (sample_size * binsizes)

    return dens, edges, err

    
def plot_histogram(energies):
    ''' Display a plot of the pyplot binned data, and self-binned data '''
    dens, edges, err = bin_data_adaptive(energies)
    
    f, (ax1, ax2) = pyplot.subplots(2, 1, figsize=(12, 18))
    # NB not corrected for background photons, bins not cut when counts < 20!
    # So this can be used as a sanity check because it is only the binned data!
    ax1.hist(energies/numpy.mean(energies), bins=143, facecolor='b',
             label="pyplot binned", alpha=0.5, normed=True)

    ax1.plot(edges[1:], dens, lw=2, c="r",
             ls="steps-mid", label="adaptive binned")
    ax1.set_ylabel("p(x)")
    ax1.set_xlabel(r'$\frac{E}{<E>}$')
    ax1.set_title("Histogram of raw data plus adaptive binned data")
    ax1.legend(loc=1)
   
    # Riley-style plot of the binned data, but log-log plot
    ax2.plot(edges[1:], dens, lw=3, c="black",
             ls="steps-mid", label="adaptive binned")
    ax2.set_ylabel("p(x)")
    ax2.set_xlabel(r'$\frac{E}{<E>}$')
    # ax2.set_title("Log-log plot of adaptive binned data")
    ax2.set_xscale('log')
    ax2.set_yscale('log')
    ax2.legend(loc=1)

    pyplot.show()

Since Riley was kind enough to provide solutions for the previous assignment, and this assignment is similar to the last question of said assignment: why not use Riley's code as a starting point? Or in some cases simply steal Riley's code :-)... I mean, it's the teacher's code so it must be correct and this must be the way to go, right? Also, it's there, this is not a programming course but a statistics course, and nobody likes to invent the wheel all over again. Credits for quite a lot of code used troughout this assignment are mostly due to Riley, but significant changes have been made.


In the words of Riley: 
> Let's first load in the data, normalise the ~~count rate~~ **energy** values by the mean ~~count rate~~ **energy** (anticipating that it will be easier to optimize our fit with smaller x values), bin the ~~count rate~~ **energy** values into evenly-sized bins, and discard the bins in the tails of the distribution if they have < 20 counts. We can then calculate the error on the probability density of each bin based on the assumption that we can approximate the binning process as following a Poisson distribution (it actually follows a binomial distribution, since we are dealing with the probability that a given x-value is binned into the bin of interest). Then we can set error = $\sqrt{n}$, where $n$ is the number of values in the bin. This makes sense if you just notice that the variance of a Poisson distribution is the expected ~~count rate~~ **energy**.

In [ ]:
if __name__ == '__main__':
    # dens, edges, err = bin_data_adaptive(energies)
    plot_histogram(energies)

So, now you know how I binned my data :-). Mind you, the blue histogram is generated from the raw data. Here matplotlib bins the data in a fixed number of bins (not adaptive), it is not corrected for background photons and bins with counts < 20 are not ignored (which will be found in the tail of the distribution). So this can be used as a sanity check because it is only the binned data, nothing else happend. Since the background is relatively low (only 1.5 photon/GeV) one does not visually observe that difference. The second plot is generated using Riley's code and changing the axis to log-log. If we expect a power law, then the binned data should look like a straight line in a log-log plot where the slope is the power law index.

Next, we define our three models, our statistical model, write a fitting routine and output the MLEs and chisq/dof. The method used is copy-pasted from Riley's code but adjusted for the new models.

In [ ]:
# Define our three models
def simple_power_law(parm, edges):
    N_0 = parm[0]
    E_0 = parm[1]
    Gamma = parm[2]
    
    # We have given the edges which allow us to do the integration over the bin
    E_left = edges[:-1]
    E_right = edges[1:]
    binsize = E_right - E_left
    
    # Formula integrated over bin
    counts = (N_0 * E_right/(1. - Gamma) * (E_right/E_0)**(-1. * Gamma)) \
        - (N_0 * E_left/(1. - Gamma) * (E_left/E_0)**(-1. * Gamma))
    
    # Now transform to density instead of counts
    dens = counts / binsize
    return dens


def broken_power_law(parm, edges):
    N_0 = parm[0]
    E_0 = parm[1]
    Gamma_1 = parm[2]
    Gamma_2 = parm[3]
    E_bk = parm[4]
    
    # We have given the edges which allow us to do the integration over the bin
    E_left = edges[:-1]
    E_right = edges[1:]
    binsize = E_right - E_left
    E_center = (E_left + E_right)/2.
    
    E_below_left = E_left[E_center <= E_bk]
    E_below_right = E_right[E_center <= E_bk]
    E_above_left = E_left[E_center > E_bk]
    E_above_right = E_right[E_center > E_bk]
    
    # Formula integrated over bin
    counts_below = \
        (N_0 * E_below_right/(1. - Gamma_1)\
        * (E_below_right/E_0)**(-1. * Gamma_1)) \
        - (N_0 * E_below_left/(1. - Gamma_1) \
        * (E_below_left/E_0)**(-1. * Gamma_1))
        
    counts_above = \
        (N_0 * (E_bk/E_0)**(-1. * Gamma_1) * E_above_right/(1. - Gamma_2)\
        * (E_above_right/E_bk)**(-1. * Gamma_2)) \
        - (N_0 * (E_bk/E_0)**(-1. * Gamma_1) * E_above_left/(1. - Gamma_2)\
        * (E_above_left/E_bk)**(-1. * Gamma_2))
        
    counts = numpy.concatenate([counts_below, counts_above])

    # Now transform to density instead of counts
    dens = counts / binsize
    return dens


def exponentially_cut_off_power_law(parm, edges):
    N_0 = parm[0]
    E_0 = parm[1]
    Gamma = parm[2]
    E_cut = parm[3]

    # We have given the edges which allow us to do the integration over the bin
    E_left = edges[:-1]
    E_right = edges[1:]
    binsize = E_right - E_left
    
    counts = (-1. * N_0 * (E_right/E_0)**(-1. * Gamma)\
        * E_right * scipy.special.expn(Gamma, E_right/E_cut)) \
        - (-1. * N_0 * (E_left/E_0)**(-1. * Gamma) \
        * E_right * scipy.special.expn(Gamma, E_left/E_cut))
    
    # Now transform to density instead of counts
    dens = counts / binsize
    return dens


# Define the statistical model, in this case we shall use a chi-squared distribution, assuming normality in the errors
def stat(parm, edges, y, dy, dist):
    if dist == "simple":
        ymod = simple_power_law(parm, edges)
    elif dist == "broken":
        ymod = broken_power_law(parm, edges)
    elif dist == "exp_cut":
        ymod = exponentially_cut_off_power_law(parm, edges)
    else:
        print "This function is not defined....choose another"
        return None
    
    X = sum((y - ymod)**2 / dy**2)
    return(X)


def fit_observed_spectrum_with_continuum_models(energies, find_emissionline=False):
        # change nr_of_bins to an integer value to change the number of bins.
        dens, edges, err = bin_data_adaptive(energies)
        
        # define an array of different dist names
        dists = ["simple", "broken", "exp_cut"]
        cols = ["red", "green", "blue"]
        results = {}
        
        if find_emissionline:
            dists = ["exp_cut_gauss"]

        fig, ((ax1, ax2), (ax3, ax4)) = pyplot.subplots(2, 2, figsize=(12, 9))
        for i in xrange(len(dists)):
            if dists[i] == "simple":
                # [N_0, E_0, Gamma]
                parm = [1.1, 1.5, 2.0]
            elif dists[i] == "broken":
                # [N_0, E_0, Gamma_1, Gamma_2, E_bk]
                parm = [1.2, 1.3, 2.0, 2.0, 3.0]
            elif dists[i] == "exp_cut":
                # [N_0, E_0, Gamma, E_cut]
                parm = [1.3, 2.2, 1.5, 1.0]
            elif dists[i] == "exp_cut_gauss":
                # [N_0, E_0, Gamma, E_cut]
                parm = [1.3, 2.2, 1.5, 1.0]
                
            result = scipy.optimize.minimize(stat, parm, args=(edges, dens, err, dists[i]),
                                             method='Nelder-Mead')
            results[dists[i]] = result
            
            ml_vals = result["x"]
            ml_func = result["fun"]
            dof = len(dens) - len(parm)
            print "[MLEs], chisq/dof:", ml_vals, ml_func/dof
            ch = scipy.stats.chi2(dof)
            pval = 1.0 - ch.cdf(ml_func)
            print "The p-value for the {0} distribution is: {1:.5f}".format(dists[i], pval)
            if dists[i] == "simple":
                ax1.plot(edges[1:], simple_power_law(ml_vals, edges), c=cols[i],
                        label=dists[i],  drawstyle="steps-mid")
            elif dists[i] == "broken":
                ax2.axvline(ml_vals[4], ls="dashed", c=cols[i])
                ax2.plot(edges[1:], broken_power_law(ml_vals, edges), c=cols[i],
                        label=dists[i], drawstyle="steps-mid")
            elif dists[i] == "exp_cut":
                ax3.axvline(ml_vals[3], ls="dashed", c=cols[i])
                ax3.plot(edges[1:], exponentially_cut_off_power_law(ml_vals, edges),
                        label=dists[i], c=cols[i], drawstyle="steps-mid")
        
        for i, ax in enumerate([ax1, ax2, ax3]):
            ax.plot(edges[1:], dens, lw=1, color="black",
                    linestyle="steps-mid")
            ax.set_ylabel("p(x)")
            ax.set_xlabel(r'$\frac{E}{<E>}$')
            ax.set_xscale('log')
            ax.set_yscale('log')
            # ax.tick_params(labelsize=15)
            # ax.legend()
            ax.set_title(dists[i])
        fig.tight_layout()
        fig.delaxes(ax4)
        pyplot.draw()
        pyplot.show()
        
        return results


if __name__ == '__main__':
    fit_results = fit_observed_spectrum_with_continuum_models(energies)

Again, in the words of Riley: 
> Now we run through a loop of these model functions and fit each one in turn to obtain the MLEs and p-values based on the best-fit model evaluation ($\chi^2$) value. The results along with plots against the data are seen ~~below~~ ** above**. (...) Notice we have actually shown the binned model in the plot, since it represents the procedure best - we do not know how the data behaves throughout any given bin, and although we have somewhat taken this into account by defining our model values as the probability density distribution ~~averaged~~ **integrated** over the bin, showing the binned model values presents the results in the clearest way. 

Note that in the above plots we have indicated the break- or cutoff energy as a vertical, dashed line. In this case the number of bins (equally spaced bins) effects the $p$-value. In some cases the chosen number of bins results in a found break- or cutoff energy that lies outside the power law (so this is probably not a good fit as it is equal to the simple power law). However, it is not good practice to use the number of bins as a fit value, (i.e. optimising the number of bins to obtained the highest $p$-value), so we will use the adaptive binsizes.

The next step would be to obtain confidence intervals on the obtained MLE's. In this case, however, we notice that the $p$-values are so low that calculating confidence intervals would make little to no sense whatoever. Nontheless, we were... inspired... again by Riley's method to use the `scipy.optimize.curve_fit` routine to calculate confidence intervals and to plot the residuals. In the words of Riley:
> (...) although this routine does not return the actual value of the statistic, it does return the Hessian, and we can simply obtain the statistic ourselves afterwards. This is shown below. It is worth noting that the p-value would change quite a bit if we were to change the number of bins.

In [ ]:
def simple_power_law_wrapper(edges, parm0, parm1, parm2):
    N_0 = parm0
    E_0 = parm1
    Gamma = parm2

    return simple_power_law((parm0, parm1, parm2), edges)

def broken_power_law_wrapper(edges, parm0, parm1, parm2, parm3, parm4):
    N_0 = parm0
    E_0 = parm1
    Gamma_1 = parm2
    Gamma_2 = parm3
    E_bk = parm4
    
    return broken_power_law((parm0, parm1, parm2, parm3, parm4), edges)


def exponentially_cut_off_power_law_wrapper(edges, parm0, parm1, parm2, parm3):
    N_0 = parm0
    E_0 = parm1
    Gamma = parm2
    E_cut = parm3

    return exponentially_cut_off_power_law((parm0, parm1, parm2, parm3), edges)


def print_confidence_intervals_and_plot_residuals(model, result, print_cis=True, plot_residuals=True):
    dens, edges, err = bin_data_adaptive(energies)

    # Conifdence intervals
    functionpicker = {"simple": simple_power_law_wrapper ,
                      "broken": broken_power_law_wrapper,
                      "exp_cut": exponentially_cut_off_power_law_wrapper}
    function = functionpicker.get(model, None)
    
    if not function:
        print "Error: incorrect model name used"
        return
    
    ml_vals = result["x"]
    moddof = len(ml_vals)
    
    if print_cis:
        ml_func = result["fun"]
        dof = len(dens) - moddof
        
        print "Results for the '{0}' model:".format(model)
        print "  Using scipy.optimize.minimze to minimize chi^2 yields:"
        print "    [MLEs], chisq/dof:", ml_vals, ml_func/dof
        print
    
    ml_vals, ml_covar = scipy.optimize.curve_fit(function, edges, dens, p0=ml_vals, sigma=err,)
    ml_funcval = stat(ml_vals, edges, dens, err, model)

    dof = len(dens) - moddof
    errs = numpy.sqrt(numpy.diag(ml_covar))
    if print_cis:
        print "  Using scipy.optimize.curve_fit to obtain confidence intervals yields:"
        print "    N_0 = {0:.3f} +/- {1:.3f}".format(ml_vals[0], err[0])
        print "    E_0 = {0:.3f} +/- {1:.3f}".format(ml_vals[1], err[1])
        print "    {parm2_name} = {0:.3f} +/- {1:.3f}".format(ml_vals[2], err[2],
            parm2_name="Gamma" if model in ["simple", "exp_cut"] else "Gamma_1")
        if moddof > 3:
            print "    {parm3_name} = {0:.3f} +/- {1:.3f}".format(ml_vals[3], err[3],
                parm3_name="Gamma_2" if model=="broken" else "E_cut")
        if moddof > 4:
            print "    E_bk = {0:.3f} +/- {1:.3f}".format(ml_vals[4], err[4])
        
        ch = scipy.stats.chi2(dof)
        pval = 1.0 - ch.cdf(ml_funcval)
        print "    p-value for this fit = {0:.5f}".format(pval)
        
    # Residuals plot
    functionpicker = {"simple": simple_power_law,
                      "broken": broken_power_law,
                      "exp_cut": exponentially_cut_off_power_law}
    function = functionpicker.get(model, None)

    if not function:
        print "Error: incorrect model name used"
        return
    
    if plot_residuals:
        binsize = numpy.array([edges[i+1] - edges[i] for i in range(len(dens))])
        
        colourpicker = {"simple": "red" , "broken": "green", "exp_cut": "blue"}
        col = colourpicker.get(model, "black")

        fig = pyplot.subplots(2, 1, figsize=(12, 9))
        gs1 = matplotlib.gridspec.GridSpec(3, 3)
        gs1.update(hspace=0)
        ax1 = pyplot.subplot(gs1[:-1,:])
        ax2 = pyplot.subplot(gs1[-1,:])
        # ax1.tick_params(labelsize=15)
        # ax2.tick_params(labelsize=15)
        # ax1.tick_params(labelbottom='off')

        ax1.set_xlim(0.2, 5.1)
        ax2.set_xlim(0.2, 5.1)

        ax1.set_ylabel("p(x)")
        ax2.set_ylabel("residuals")
        ax2.set_xlabel(r'$\frac{E}{<E>}$')
        ax1.errorbar(edges[1:] + binsize/2., dens, xerr=binsize/2., yerr=err,
                     label="data", c="black", marker='o', linestyle='', markersize=3)
        ax1.plot(edges[1:] + binsize/2., function(ml_vals, edges),
                 label=model, c=col, drawstyle='steps-mid')
        
        if model == "broken":
            break_or_cut = ml_vals[4]
            ax1.axvline(break_or_cut, ls="dashed", c=col)
        elif model == "exp_cut":
            break_or_cut = ml_vals[3]
            ax1.axvline(break_or_cut, ls="dashed", c=col)

        ax2.axhline(y=0, linewidth=2, linestyle='dashed', c="black")

        ax2.errorbar(edges[1:] + binsize/2., dens - function(ml_vals, edges), yerr=err,
                     c="black", drawstyle="steps-mid")
        
        ax1.legend()
        
        # fix overlapping ticks
        ax2.set_ylim(ymax=0.099)
        ax1.tick_params(labelbottom='off')
        nbins = len(ax2.get_yticklabels())
        ax2.yaxis.set_major_locator(MaxNLocator(nbins=nbins, prune='upper'))
        
        # log-log
        ax1.set_xscale('log')
        ax2.set_xscale('log')
        ax1.set_yscale('log')
        
        pyplot.show()


if __name__ == '__main__':
    for model, result in fit_results.iteritems():
        if model != "exp_cut":
            pass  # use continue if we only want to show the best-fit model: exp_cut; use pass to show all.
        print_confidence_intervals_and_plot_residuals(model, result)
        print 


In the words of Riley:
> The plot above shows the results of the optimization.

We can see from the residuals that in most of the bins the model do not fit the data within the errors above the break- or cut_off energy which makes sense given the (very low) obtained p-value of 0.00000. 

### Question 3
- Search for emission lines, which may carry crucial information if they are associated with specific dark matter particle decays. You may assume a Gaussian profile to fit the emission line:
$$ dN = \frac{N_{\rm line}}{\sigma \sqrt{2 \pi}} \exp\left( \frac{(E - E_{\rm cent})^2)}{2 \sigma^2} \right) dE $$ where $E_{\rm cent}$ is the line centroid energy, $\sigma$ is the line width and $N_{\rm line}$ is the expected total number of photons contained in the line. The width of any lines is set by the instrumental resolution (which is a function of energy) and is given by:

$$ \sigma = 2.0 \sqrt{\frac{E_{\rm cent}}{200 \, \rm{ GeV}}} \, \rm{ Gev} $$

### Answer 3

In [ ]:
def exponentially_cut_off_power_law(parm, x, dE):
    N_0 = parm[0]
    E_0 = parm[1]
    Gamma = parm[2]
    E_cut = parm[3]  # Is this a fit parameter, though?
    N_line = parm[4]
    E_cent = parm[5]
    E = x
    
    dN = N_0 * (E/E_0)**(-1. * Gamma) * numpy.exp(-1. * E/E_cut) * dE
    emission_line = 
    return dN


def gauss_wrapper(dE):
    def gauss(x, parm0, parm1):
        N_line = parm0
        E_cent = parm1  # GeV
        sigma = 2.0 * numpy.sqrt(E_cent / 200)  # GeV
        E = x

        dN = N_line / (sigma * numpy.sqrt(2.*numpy.pi)) * numpy.exp(1.*(E - E_cent)**2/(2.*sigma**2)) * dE
        return dN
    return gauss

if __name__ == '__main__':


### Question 4
- [Professor Denzil Dexter](http://www.teachertube.com/video/professor-denzil-dexter-81357) of the University of Southern California has proposed a dark matter candidate particle, the _Dextron_, which should result in a Gaussian emission line of width 1 GeV at an energy of 45.3 GeV. In case this line isn't detected, use your data to set a $3\sigma$ upper limit on the predicted number of photons that this line could contain.

Note that in all cases you should estimate $p$-values and significances for your hypothesis tests. You should explain your reasoning and choices and discuss how you interpret your results as you go along. Make sure that you make sensible decisions about how your report your results, e.g. about the number of significant figures used. You should also, in as much as possible, determine confidence intervals on the best-fitting model parameters (for continuum and emission line models). If errors appear to be correlated between parameters, you should also attempt to plot 2-dimensional confidence contours.Overall, we are looking for evidence that you understand and can apply the material in the course on model-fitting, parameter estimation and hypothesis testing, so you will find the information in the lectures in weeks 4, 5 and 6 to be very useful, as well as the case studies lecture from week 6.

### Answer 4